In [1]:
import pandas as pd
import numpy as np
import sys,re, pickle, nltk
from nltk import word_tokenize
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Binarizer

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adrienawong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df = pd.read_csv('../Outputs/Summary/article_summary-202001.csv') #reads the summary data

In [4]:
len(df)

19815

In [5]:
# Splits the string and converts it into a list
df['Article Topics Distributions'] = df['Article Topics Distributions'].apply(lambda x:x[1:-1].split(','))
df['Articles Vector Centroid'] = df['Article Vector Centroid'].apply(lambda x:x[1:-1].split())
df

,Unnamed: 0,Snapshot,Article,Article Vector Centroid,Article Topics Distributions,TextStat Fleisch Reading Difficulty,Degree,Betweenness Centrality,Clicks in month,Article Length,Target Complexity,Articles Vector Centroid
0,0,202001,AccessibleComputing,[],[],206.84,NaN,NaN,NaN,2,NaN,[]
1,1,202001,Anarchism,[-0.711271 -0.10975295 -0.55753543 0.434648...,"[nan, 0.7517666220664978, 0.2385802417993545...",-3604.54,2.0,0.017757,82359.0,40949,NaN,"[-0.711271, -0.10975295, -0.55753543, 0.434648..."
2,2,202001,AfghanistanHistory,[],[],206.84,NaN,NaN,NaN,2,NaN,[]
3,3,202001,AfghanistanGeography,[],[],206.84,NaN,NaN,NaN,2,NaN,[]
4,4,202001,AfghanistanPeople,[],[],206.84,NaN,NaN,NaN,2,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
19810,19810,202001,The Lord of the Rings/One Ring,[],[],206.84,NaN,NaN,NaN,2,NaN,[]
19811,19811,202001,Tax Freedom Day,[-4.66263973e-01 -5.26436425e-02 -4.48773343e-...,"[nan, 0.08460813760757446, 0.811170220375061...",-1044.02,NaN,NaN,NaN,11025,NaN,"[-4.66263973e-01, -5.26436425e-02, -4.48773343..."
19812,19812,202001,Tax,[-4.03108368e-01 1.17827849e-01 -4.26096367e-...,"[nan, 0.02796074002981186, 0.683391809463501...",-1123.87,136.0,0.008156,50101.0,12139,NaN,"[-4.03108368e-01, 1.17827849e-01, -4.26096367e..."
19813,19813,202001,Transhumanism,[-0.58925328 -0.05426568 -0.51863559 0.428650...,"[nan, 0.6766897439956665, 0.0286630094051361...",-3908.02,193.0,0.020455,36725.0,45063,NaN,"[-0.58925328, -0.05426568, -0.51863559, 0.4286..."


In [6]:
# Splits the vectors into multiple columns
df[['ATD_1','ATD_2','ATD_3', 'ATD_4','ATD_5']] = pd.DataFrame(df['Article Topics Distributions'].values.tolist(), index= df.index).replace('nan', '0').replace(' nan', '0').apply(pd.to_numeric)
df_avc = pd.DataFrame(df['Articles Vector Centroid'].values.tolist(), index= df.index).add_prefix('AVC_')
df = pd.concat([df, df_avc[:]], axis=1)
df = df.drop(['Article Topics Distributions','Articles Vector Centroid'],axis=1)

In [7]:
df['Degree'].fillna(0, inplace=True)
df['Betweenness Centrality'].fillna(0, inplace=True)
df['Clicks in month'].fillna(0, inplace=True)
df = df[df['Article Vector Centroid'] != '[]']

In [8]:
df_train = df[df['Target Complexity'] >= 0]
df_train

,Unnamed: 0,Snapshot,Article,Article Vector Centroid,TextStat Fleisch Reading Difficulty,Degree,Betweenness Centrality,Clicks in month,Article Length,Target Complexity,...,AVC_90,AVC_91,AVC_92,AVC_93,AVC_94,AVC_95,AVC_96,AVC_97,AVC_98,AVC_99
124,124,202001,Appellate procedure in the United States,[-6.48681880e-01 -1.84573946e-01 -6.03912114e-...,-1915.57,0.0,0.0,0.0,20512,2.0,...,4.40493375e-01,6.22147808e-01,4.17365160e-01,2.14334000e-01,-4.12655922e-01,4.20328349e-01,-6.21038972e-01,2.64847956e-01,-6.99125810e-01,2.90836128e-01
169,169,202001,Politics of Angola,[-0.56815767 0.31761419 -0.5509628 0.414295...,-615.70,0.0,0.0,0.0,6997,1.0,...,0.53496912,0.64634101,0.01733846,0.02766238,-0.5899628,0.31402331,-0.39659695,0.28226344,-0.6706099,0.2489592
187,187,202001,Algorithms (journal),[-6.90959530e-01 6.62405149e-03 -5.43830507e-...,-80.80,0.0,0.0,0.0,878,1.0,...,2.47249754e-02,5.13999651e-01,1.08502442e-01,-3.53509678e-01,-4.98272901e-01,4.50111098e-02,-3.75050277e-01,-2.25549528e-04,-8.65985038e-01,7.00272436e-02
200,200,202001,Abstract (law),[-0.48134389 -0.20785407 -0.77905337 0.190248...,-121.39,0.0,0.0,0.0,1563,1.0,...,0.38722021,0.64966955,0.28937542,0.0667619,-0.34524085,0.44446965,-0.57143962,-0.03183226,-0.9344908,-0.10813795
276,276,202001,Anna Kournikova,[-0.85636401 -0.37553962 -0.37182447 0.303480...,-1251.76,0.0,0.0,0.0,13321,1.0,...,0.37031487,0.9607843,0.15653826,0.09651409,-0.43298771,0.05701767,-0.1275363,0.25753235,-0.48354847,0.05648137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19607,19607,202001,Mort,[-0.66011735 -0.41612261 -0.24327076 0.453251...,-600.13,1.0,0.0,5115.0,6573,1.0,...,0.29937679,0.70934273,0.09137015,0.29327762,-0.35836303,0.19285132,-0.30693285,0.01529428,-0.42521398,0.00558949
19608,19608,202001,Tim Berners-Lee,[-0.63567759 -0.15243937 -0.45719599 0.655058...,-562.58,0.0,0.0,0.0,6182,2.0,...,0.0856331,0.79227499,0.13782312,-0.28932595,-0.61079304,0.04543556,-0.3207884,0.05378748,-0.67645106,-0.14859692
19635,19635,202001,The Big Test,[-1.15534190e+00 -4.92014409e-01 -2.32725336e-...,2.79,0.0,0.0,0.0,270,1.0,...,2.82657408e-01,1.13086985e+00,1.96053938e-01,-4.63979509e-01,-3.66391148e-01,-3.02888739e-02,-9.34123635e-01,-4.23972815e-01,-5.37898550e-01,-7.10542477e-02
19636,19636,202001,Tradition,[-0.6623432 -0.2833228 -0.58263032 0.394649...,-1024.41,1.0,0.0,23645.0,11581,2.0,...,0.3949872,0.59205561,0.24242471,0.449114,-0.7861806,0.35199775,-0.5968961,-0.20049066,-0.97563727,-0.14527617


In [9]:
def targetify(col):
    if col == 2 or col == 3:
        return 1
    else:
        return 0

In [10]:
df_train['Target Complexity'] = df['Target Complexity'].apply(targetify)

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
features  = [c for c in df.columns]
features.remove('Unnamed: 0')
features.remove('Snapshot')
features.remove('Article')
features.remove('Article Vector Centroid')
features.remove('Target Complexity')

In [12]:
X = df_train[features]
y = df_train['Target Complexity']

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X[['Article Length', 'Clicks in month']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [14]:
X[['Article Length', 'Clicks in month']] = scaler.transform(X[['Article Length', 'Clicks in month']])

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [15]:
model = LogisticRegression().fit(X, y)

/Users/adrienawong/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
pickle.dump(model, open('../Outputs/model/complexity_model.sav', 'wb'))

In [17]:
df[['Article Length', 'Clicks in month']] = scaler.transform(df[['Article Length', 'Clicks in month']])
df['proba'] =  model.predict_proba(df[features])[:,1]

In [18]:
df_sorted = df.sort_values(by='proba', ascending=False)

In [19]:
df_sorted.to_csv('../Outputs/probabilities.csv')